In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer, KNNImputer
import numpy as np

folder_path = "../data/"

In [3]:
def get_df_and_truth(data_list, truth_list, cols=['Chr', 'START_POS_REF', 'END_POS_REF'], split=False, extra='-small'):
    
    df_res = []
    truth_res = {}
    for data_name, truth_name in zip(data_list, truth_list):
        data_path = folder_path + f"snv-parse-{data_name}{extra}.txt"
        df = pd.read_csv(data_path, sep='\t', low_memory=False)
        df['is_real'] = data_name[:4] == 'real'
        df['ds_name'] = data_name
        df['ds_type'] = data_name[:5]
        truth_path = folder_path + f"{data_name}/{truth_name}.bed"
        truth = pd.read_csv(truth_path, sep='\t', header=None, names = ['Chr', 'START_POS_REF', 'END_POS_REF'])
        truth['label'] = 1
        df_res.append(df)
        truth_res[data_name] = truth

    return df_res, truth_res

In [4]:
data_list = ["real1", "real2_part1", "syn1", "syn2", "syn3", "syn4", "syn5"]
#data_list = ["real1", "real2_part1"]
truth_list = ["real1_truth", "real2_truth_chr1to5", "syn1_truth", "syn2_truth", "syn3_truth", "syn4_truth", "syn5_truth" ]
#truth_list = ["real1_truth", "real2_truth_chr1to5"]
df_list, truth_dict = get_df_and_truth(data_list, truth_list, extra="-small")
df_list_large, truth_dict = get_df_and_truth(data_list, truth_list, extra="")

FileNotFoundError: [Errno 2] No such file or directory: '../data/snv-parse-real1-small.txt'

In [ ]:
merged_list = [df.merge(truth, how='left').fillna({'label':0}) for df, truth in zip(df_list, truth_dict.values())]
merged_list_large = [df.merge(truth, how='left').fillna({'label':0}) for df, truth in zip(df_list_large, truth_dict.values())]

In [ ]:
def get_num_in_set(df, truth):
    return df.merge(truth, how='left')['label'].fillna(0).sum() 

In [ ]:
for i, name in enumerate(data_list):
    truth = truth_dict[name]
    tmp = 'label'
    print(f'{name}, {merged_list[i][tmp].sum()}, {merged_list_large[i][tmp].sum()}, {len(truth)}')

real1, 1274.0, 1308.0, 1319
real2_part1, 449.0, 462.0, 491
syn1, 3520.0, 3522.0, 3537
syn2, 4303.0, 4313.0, 4332
syn3, 7623.0, 7767.0, 7903
syn4, 14343.0, 15389.0, 16315
syn5, 44703.0, 45088.0, 45383


In [ ]:
# 
name = 'real1'
truth = truth_dict[name]
true_values = merged_list[0].loc[merged_list[0]['label'] == 1]
true_values_large = merged_list_large[0].loc[merged_list_large[0]['label'] == 1]
true_values.shape, true_values_large.shape

((1274, 61), (1308, 61))

In [ ]:
true_values_large.loc[~true_values_large['START_POS_REF'].isin(true_values['START_POS_REF'])][['FILTER_Mutect2_data', 'FILTER_Varscan_data', 'FILTER_Vardict_data', 'FILTER_Freebayes_data']]

,FILTER_Mutect2_data,FILTER_Varscan_data,FILTER_Vardict_data,FILTER_Freebayes_data
198342,clustered_events;multi_event_alt_allele_in_normal,NaN,P0.01Likely,NaN
365653,t_lod_fstar,NaN,NaN,NaN
554294,t_lod_fstar,NaN,NaN,NaN
936697,t_lod_fstar,NaN,NaN,NaN
1000040,t_lod_fstar,NaN,NaN,NaN
1183643,t_lod_fstar,NaN,NaN,NaN
1213801,t_lod_fstar,NaN,NaN,NaN
1223353,t_lod_fstar,NaN,NaN,NaN
1258561,t_lod_fstar,NaN,NaN,NaN
1330264,clustered_events,NaN,NaN,NaN


In [ ]:
# 
name = 'real2_part1'
i=1
truth = truth_dict[name]
true_values = merged_list[i].loc[merged_list[i]['label'] == 1]
true_values_large = merged_list_large[i].loc[merged_list_large[i]['label'] == 1]
true_values_large.loc[~true_values_large['START_POS_REF'].isin(true_values['START_POS_REF'])][['FILTER_Mutect2_data', 'FILTER_Varscan_data', 'FILTER_Vardict_data', 'FILTER_Freebayes_data']]

NameError: name 'truth_dict' is not defined

In [ ]:
true_values.columns

Index(['Chr', 'START_POS_REF', 'END_POS_REF', 'REF', 'ALT', 'REF_MFVdVs',
       'ALT_MFVdVs', 'Sample_Name', 'FILTER_Mutect2', 'FILTER_Freebayes',
       'FILTER_Vardict', 'FILTER_Varscan', 'FILTER_Mutect2_data',
       'FILTER_Freebayes_data', 'FILTER_Vardict_data', 'FILTER_Varscan_data',
       'm2_ClippingRankSum', 'm2_DP', 'm2_ECNT', 'm2_FS', 'm2_MQ', 'm2_MQ0',
       'm2_MQRankSum', 'm2_ReadPosRankSum', 'f_AN', 'f_DP', 'f_DPB', 'f_EPPR',
       'f_GTI', 'f_MQMR', 'f_NS', 'f_NUMALT', 'f_ODDS', 'f_PAIREDR', 'f_PQR',
       'f_PRO', 'f_QR', 'f_RO', 'f_RPPR', 'f_SRF', 'f_SRP', 'f_SRR', 'vs_DP',
       'vs_GPV', 'vs_SPV', 'vs_SS', 'vs_SSC', 'vd_DP', 'vd_MSI', 'vd_MSILEN',
       'vd_SAMPLE', 'vd_SHIFT3', 'vd_SOR', 'vd_SSF', 'vd_STATUS', 'vd_TYPE',
       'vd_VD', 'is_real', 'ds_name', 'ds_type', 'label'],
      dtype='object')